In [1]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
pretrained_model = "distilbert-base-uncased"
pretrained_model = "xlm-roberta-large"
pretrained_model = "microsoft/deberta-v3-small"
# pretrained_model = "kamalkraj/deberta-v2-xlarge"
# model_checkpoint = "/root/ner-model/test-ner/checkpoint-2500/"

source_dataset_name = "conll2003"
model_checkpoint = f"../data/models/{pretrained_model}/{source_dataset_name}"

use_pretrained_model = True
if use_pretrained_model:
    model_checkpoint = pretrained_model

batch_size = 16

# dataset_name = "wnut_17"
dataset_name = "conll2003"

# Loading dataset

In [2]:
from datasets import load_dataset, load_metric

In [3]:
datasets = load_dataset(dataset_name)

Found cached dataset conll2003 (/root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [6]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [7]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [8]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(datasets["train"])

,id,tokens,pos_tags,chunk_tags,ner_tags
0,13002,"[Morris, declined, to, address, the, allegations, directly, in, his, resignation, statement, but, said, he, was, quitting, to, avoid, becoming, a, campaign, issue, .]","[NNP, VBD, TO, VB, DT, NNS, RB, IN, PRP$, NN, NN, CC, VBD, PRP, VBD, VBG, TO, VB, VBG, DT, NN, NN, .]","[B-NP, B-VP, I-VP, I-VP, B-NP, I-NP, B-ADVP, B-PP, B-NP, I-NP, I-NP, O, B-VP, B-NP, B-VP, I-VP, I-VP, I-VP, I-VP, B-NP, I-NP, I-NP, O]","[B-PER, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,1956,"[N., Hussain, c, Saeed, Anwar, b, Waqar, Younis, 12]","[NNP, NNP, SYM, NNP, NNP, SYM, NNP, NNP, CD]","[B-NP, I-NP, O, B-NP, I-NP, O, B-NP, I-NP, I-NP]","[B-PER, I-PER, O, B-PER, I-PER, O, B-PER, I-PER, O]"
2,11110,"[Zambian, President, Frederick, Chiluba, shuffled, his, cabinet, on, Wednesday, to, fill, a, vacancy, left, after, the, sacking, of, Legal, Affairs, Minister, Remmy, Mushota, .]","[NNP, NNP, NNP, NNP, VBD, PRP$, NN, IN, NNP, TO, VB, DT, NN, VBN, IN, DT, NN, IN, NNP, NNP, NNP, NNP, NNP, .]","[B-NP, I-NP, I-NP, I-NP, B-VP, B-NP, I-NP, B-PP, B-NP, B-VP, I-VP, B-NP, I-NP, B-VP, B-PP, B-NP, I-NP, B-PP, B-NP, I-NP, I-NP, I-NP, I-NP, O]","[O, O, B-PER, I-PER, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PER, I-PER, O]"
3,13645,"[The, girl, was, taken, to, hospital, .]","[DT, NN, VBD, VBN, TO, NN, .]","[B-NP, I-NP, B-VP, I-VP, B-PP, B-NP, O]","[O, O, O, O, O, O, O]"
4,13940,"[BARRY, ,, Wales, 1996-08-30]","[NNP, ,, NNP, CD]","[B-NP, O, B-NP, I-NP]","[B-LOC, O, B-LOC, O]"
5,481,"[Second, round]","[JJ, NN]","[B-NP, I-NP]","[O, O]"
6,9825,"["", Although, the, final, draft, of, the, treaty, probably, did, n't, totally, satisfy, any, country, ,, it, was, in, general, balanced, ,, "", the, official, People, 's, Daily, newspaper, said, in, a, commentary, .]","["", IN, DT, JJ, NN, IN, DT, NN, RB, VBD, RB, RB, VB, DT, NN, ,, PRP, VBD, IN, JJ, JJ, ,, "", DT, JJ, NNS, POS, JJ, NN, VBD, IN, DT, NN, .]","[O, B-SBAR, B-NP, I-NP, I-NP, B-PP, B-NP, I-NP, B-ADVP, B-VP, I-VP, I-VP, I-VP, B-NP, I-NP, O, B-NP, B-VP, B-PP, B-NP, I-NP, I-NP, O, B-NP, I-NP, I-NP, B-NP, I-NP, I-NP, B-VP, B-PP, B-NP, I-NP, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, O, O, O, O, O, O]"
7,13858,"[INTEREST, RATES, Latest, rate, Pvs, rate, Date, changed]","[NN, NNS, JJS, NN, VBZ, NN, NNP, VBD]","[B-NP, I-NP, I-NP, I-NP, B-VP, B-NP, I-NP, B-VP]","[O, O, O, O, O, O, O, O]"
8,2252,"[Evolution, of, the, women, 's, 1,000]","[NN, IN, DT, NNS, POS, CD]","[B-NP, B-PP, B-NP, I-NP, B-NP, I-NP]","[O, O, O, O, O, O]"
9,13522,"[One, soldier, was, discharged, because, of, "", mental, incapacity, and, inebetude, (, sic, ), of, the, brain, ,, alleged, by, the, Patient, to, be, connected, with, a, fall, on, the, head, but, believed, to, arise, from, long, continued, and, excessive, masturbation, ., ""]","[CD, NN, VBD, VBN, RB, IN, "", JJ, NN, CC, NN, (, JJ, ), IN, DT, NN, ,, JJ, IN, DT, NNP, TO, VB, VBN, IN, DT, NN, IN, DT, NN, CC, VBD, TO, VB, IN, JJ, VBD, CC, JJ, NN, ., ""]","[B-NP, I-NP, B-VP, I-VP, B-PP, I-PP, O, B-NP, I-NP, O, B-NP, O, B-ADJP, O, B-PP, B-NP, I-NP, O, B-NP, B-PP, B-NP, I-NP, B-VP, I-VP, I-VP, B-PP, B-NP, I-NP, B-PP, B-NP, I-NP, O, B-VP, I-VP, I-VP, B-PP, B-NP, B-VP, O, B-NP, I-NP, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


# Preprocessing dataset

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [12]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [1, 5365, 261, 291, 269, 311, 4378, 300, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
example = datasets["train"][4]
print(example["tokens"])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


In [14]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '▁Germany', "▁'", 's', '▁representative', '▁to', '▁the', '▁European', '▁Union', "▁'", 's', '▁veterinary', '▁committee', '▁Werner', '▁Z', 'wing', 'mann', '▁said', '▁on', '▁Wednesday', '▁consumers', '▁should', '▁buy', '▁sheep', 'meat', '▁from', '▁countries', '▁other', '▁than', '▁Britain', '▁until', '▁the', '▁scientific', '▁advice', '▁was', '▁clearer', '▁.', '[SEP]']


In [15]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(31, 38)

In [16]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


In [17]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

38 38


In [18]:
label_all_tokens = True

In [19]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [20]:
tokenize_and_align_labels(datasets['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[1, 2805, 27144, 2324, 660, 264, 20007, 1668, 12649, 323, 2], [1, 2557, 28080, 2], [1, 98641, 4773, 271, 4425, 271, 2944, 2], [1, 279, 1559, 2653, 357, 277, 1561, 278, 26748, 275, 2324, 1678, 264, 2888, 264, 48371, 1668, 12649, 583, 3959, 2003, 786, 6943, 9925, 1682, 295, 282, 10364, 264, 8260, 323, 2], [1, 2439, 382, 268, 4290, 264, 262, 1559, 2432, 382, 268, 15275, 3066, 31127, 2548, 7562, 8556, 357, 277, 1623, 2888, 403, 809, 8260, 34539, 292, 1226, 340, 354, 3491, 583, 262, 3122, 1678, 284, 14025, 323, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [21]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-f9c7eb8dcfe26b77.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-a9cd9cd97f81cf34.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98/cache-bd12e89ba7527ce7.arrow


# Fine-tuning the model

In [22]:
from transformers import AutoConfig, AutoModelForTokenClassification, TrainingArguments, Trainer
# config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), ignore_mismatched_sizes=True) #, from_tf=True

2022-10-07 05:11:54.723046: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-07 05:11:54.825816: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-07 05:11:55.236970: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-07 05:11:55.237015: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [23]:
args = TrainingArguments(
    # f"test-{task}",
    f"../data/models/{pretrained_model}/{dataset_name}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=0.01,
    
    save_strategy='epoch',
    save_total_limit = 3
    
)

In [24]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [25]:
metric = load_metric("seqeval")
# from evaluate import evaluator

# task_evaluator = evaluator("token-classification")

/tmp/ipykernel_1446497/1092872775.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [26]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [27]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [28]:
import torch
torch.cuda.empty_cache()

In [29]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [30]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: chunk_tags, id, pos_tags, tokens, ner_tags. If chunk_tags, id, pos_tags, tokens, ner_tags are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14041
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3512
You're using a DebertaV2TokenizerFast tokenizer.

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.183900,0.059911,0.913874,0.928824,0.921288,0.984555
2,0.040500,0.046585,0.939762,0.954218,0.946935,0.989374
3,0.022700,0.041415,0.949427,0.960322,0.954843,0.990624
4,0.015100,0.041785,0.949269,0.959468,0.954341,0.990427


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: chunk_tags, id, pos_tags, tokens, ner_tags. If chunk_tags, id, pos_tags, tokens, ner_tags are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16
Saving model checkpoint to ../data/models/microsoft/deberta-v3-small/conll2003/checkpoint-878
Configuration saved in ../data/models/microsoft/deberta-v3-small/conll2003/checkpoint-878/config.json
Model weights saved in ../data/models/microsoft/deberta-v3-small/conll2003/checkpoint-878/pytorch_model.bin
tokenizer config file saved in ../data/models/microsoft/deberta-v3-small/conll2003/checkpoint-878/tokenizer_config.json
Special tokens file saved in ../data/models/microsoft/deberta-v3-small/conll2003/checkpoint-878/special_tokens_map.json
/root/.cache/pypoetry/virtualenvs/sequence-labe

TrainOutput(global_step=3512, training_loss=0.05225417073094763, metrics={'train_runtime': 170.3342, 'train_samples_per_second': 329.728, 'train_steps_per_second': 20.618, 'total_flos': 650934903454200.0, 'train_loss': 0.05225417073094763, 'epoch': 4.0})

In [31]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: chunk_tags, id, pos_tags, tokens, ner_tags. If chunk_tags, id, pos_tags, tokens, ner_tags are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:745: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_scores = torch.bmm(query_layer, key_layer.transpose(-1, -2)) / torch.tensor(
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:829

{'eval_loss': 0.04178491234779358,
 'eval_precision': 0.9492692354149052,
 'eval_recall': 0.9594677084605054,
 'eval_f1': 0.9543412264723741,
 'eval_accuracy': 0.9904270017764326,
 'eval_runtime': 2.9727,
 'eval_samples_per_second': 1093.28,
 'eval_steps_per_second': 68.624,
 'epoch': 4.0}

In [32]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: chunk_tags, id, pos_tags, tokens, ner_tags. If chunk_tags, id, pos_tags, tokens, ner_tags are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3250
  Batch size = 16


{'LOC': {'precision': 0.9671945701357466,
  'recall': 0.9812547819433818,
  'f1': 0.9741739460691227,
  'number': 2614},
 'MISC': {'precision': 0.8797190517998245,
  'recall': 0.8820422535211268,
  'f1': 0.880879120879121,
  'number': 1136},
 'ORG': {'precision': 0.9294765840220386,
  'recall': 0.9393095768374164,
  'f1': 0.9343672112988091,
  'number': 1796},
 'PER': {'precision': 0.9745604190048635,
  'recall': 0.9848771266540642,
  'f1': 0.9796916133884919,
  'number': 2645},
 'overall_precision': 0.9492692354149052,
 'overall_recall': 0.9594677084605054,
 'overall_f1': 0.9543412264723741,
 'overall_accuracy': 0.9904270017764326}